# SVM

- 복잡한 분류 문제에 적합  

- 선형/비선형  

- 대표적인 이진 분류 모델  

- 왜 인기?  
    - 분류를 할 때 가장 최적의 선을 찾아줌(두 데이터와 선의 거리까지 고려)

- vector 선으로 분류


### 선형
- 라지 마진 분류
    - 직선으로 경계가 나눌 수 있는가?
    - 특성이 2개
    - 결정 경계가 샘플에 가까우면 안된다.
    
- 하드 마진 분류
    - 이상치(선형 분류가 되지만, 하나(이상치)가 반대쪽 영역에 가있음)에 민감하다
    - 데이터가 선형으로 분류되어야 한다.
    
- 소프트 마진 분류
    - 하드 마진 분류에 비해 이상치에 덜민감(유연한 모델)
    - 'C'(하이프파라미터)를 이용하여 균형을 조정한다.


### 비선형
##### 선형은 특성이 2개였지만, 비선형은 특성을 추가하여서 선형 구분 데이터셋을 만든다.



### 0501
overfitting 등의 문제가 생길 수 있다고 함. 아직 함수 하나하나를 이해하지는 못했음. 육안으로 봤을 때에는 PCA를 통해 차원 축소를 한 이후에 해야한다고 생각함.하지만 실제로 정확도가 더 높게 나왔다.


# 0508

- 특성 11개를 다 보지 않고 가장 연관이 있다고 판단된 특성 7개를 뽑아서 test하였다.

- 우리가 가진 data값 : 수치형(정량형)

- feature 5인 chlorides가 일정해서 사용할 수 없다는 식으로 나오는데 수가 다른 것에 비해서 작아서 그런가?

- __아무튼 ANOVA (분산 분석)의 F-값 계산해서 최고로 연관성이 있다고 나온 특성 7개만 뽑았다. quality, Binary quality 두개 다 실시__
<br>

- 그 결과 quality, Binary quality 모두에서 __특성값 11개일 때보다 정확도가 낮게__ 나왔다. RBF 커널 적용 전까지는 괜찮다가 __RBF커널을 적용하면 특성의 개수가 줄어들수록 정확도가 감소한다.__




# 추가적으로 확인해봐야할것(다음시간에..)

- overfitting등의 문제에 대해 더 찾아보자
- PCA를 이용해서 주성분분석을 하여 선형차원축소를 해보자

In [1]:
!pip install xgboost

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [2]:
!pip install --upgrade pip

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages (20.1)


In [3]:
# Data manipulation
import numpy as np
import pandas as pd

# Data pre-processing
from sklearn.preprocessing import StandardScaler as ss

# Dimensionality reduction
from sklearn.decomposition import PCA

#  Data splitting and model parameter search
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling modules
from xgboost.sklearn import XGBClassifier

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_importance

In [4]:
data=pd.read_csv("./data/binary_winequalityN.csv")
#dataw=pd.read_csv("./data/1_white.csv")
#datar=pd.read_csv("./data/1_red.csv")

data=data.dropna(axis=0)  ##없애버림, 평균값 넣으려면 axis=0대신에 inplate=True 사용
#dataw=dataw.dropna(axis=0)

print "wine 데이터셋의 크기:" ,data.shape
print "\nwine 데이터셋의 키:", data.keys()

wine 데이터셋의 크기: (6463, 14)

wine 데이터셋의 키: Index([u'type', u'fixed acidity', u'volatile acidity', u'citric acid',
       u'residual sugar', u'chlorides', u'free sulfur dioxide',
       u'total sulfur dioxide', u'density', u'pH', u'sulphates', u'alcohol',
       u'quality', u'binaryQuality'],
      dtype='object')


데이터셋의 키
['type', 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6463 entries, 0 to 6496
Data columns (total 14 columns):
type                    6463 non-null object
fixed acidity           6463 non-null float64
volatile acidity        6463 non-null float64
citric acid             6463 non-null float64
residual sugar          6463 non-null float64
chlorides               6463 non-null float64
free sulfur dioxide     6463 non-null float64
total sulfur dioxide    6463 non-null float64
density                 6463 non-null float64
pH                      6463 non-null float64
sulphates               6463 non-null float64
alcohol                 6463 non-null float64
quality                 6463 non-null int64
binaryQuality           6463 non-null int64
dtypes: float64(11), int64(2), object(1)
memory usage: 757.4+ KB


# 훈련 데이터와 테스트 데이터

In [195]:
#x_train에 적재하기 위한 2차원 배열의 특성 dataset을 만든다.
#6463*11(?) 사이즈

#실험의 편의성을 위해 data마다 이름을 붙였다.
f1=data['fixed acidity']
f2=data['volatile acidity']
f3=data['citric acid']
f4=data['residual sugar']
f5=data['chlorides']
f6=data['free sulfur dioxide']
f7=data['total sulfur dioxide']
f8=data['density']
f9=data['pH']
f10=data['sulphates']
f11=data['alcohol']


c_data=np.c_[f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11]

In [97]:
'''
X_train, X_test, y_train, y_test = train_test_split(c_data, data['quality'],random_state=0)

print "X_train 크기: ", X_train.shape #(#(numbe) of data*75%, # of features)
print "y_train 크기: ", y_train.shape #x트레인 데이터에 대한 정답
print "X_test 크기: ", X_test.shape
print "y_test 크기: ", y_test.shape
'''

'\nX_train, X_test, y_train, y_test = train_test_split(c_data, data[\'binaryQuality\'],random_state=0)\n\nprint "X_train \xed\x81\xac\xea\xb8\xb0: ", X_train.shape #(#(numbe) of data*75%, # of features)\nprint "y_train \xed\x81\xac\xea\xb8\xb0: ", y_train.shape #x\xed\x8a\xb8\xeb\xa0\x88\xec\x9d\xb8 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0\xec\x97\x90 \xeb\x8c\x80\xed\x95\x9c \xec\xa0\x95\xeb\x8b\xb5\nprint "X_test \xed\x81\xac\xea\xb8\xb0: ", X_test.shape\nprint "y_test \xed\x81\xac\xea\xb8\xb0: ", y_test.shape\n'

### 특성 삭제하기

In [196]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

In [238]:
##우리 데이터는 수치형!
fvalue_selector = SelectKBest(f_classif, k=5)
##이걸 쓰려니까 f5(염화물)이 constant하다고 나와서 f5제외했음? 될때도있네
features_kbest = fvalue_selector.fit_transform(c_data, data['quality'])

In [239]:
print"원본 특성 개수:", c_data.shape[1]
print"줄어든 특성 개수:", features_kbest.shape[1]

원본 특성 개수: 11
줄어든 특성 개수: 5


In [240]:
X_train, X_test, y_train, y_test = train_test_split(features_kbest, data['quality'],random_state=0)

print "X_train 크기: ", X_train.shape #(#(numbe) of data*75%, # of features)
print "y_train 크기: ", y_train.shape #x트레인 데이터에 대한 정답
print "X_test 크기: ", X_test.shape
print "y_test 크기: ", y_test.shape

X_train 크기:  (4847, 5)
y_train 크기:  (4847,)
X_test 크기:  (1616, 5)
y_test 크기:  (1616,)


## SVM

---

In [241]:
from sklearn.svm import SVC, LinearSVC

## SVM 분류기

OvA를 사용하여 분류

In [242]:
from sklearn.svm import LinearSVR
lin_clf = LinearSVC(max_iter=100000, random_state=42)
lin_clf.fit(X_train, y_train)

/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=100000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

#### 훈련 set에 대한 예측을 만들어 정확도를 측정

In [243]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)
##(11개)quality로 했을 때: 0.5188776562822365
##(9개)quality로 했을 때: 0.5283680627192078
##(7개)quality로 했을 때: 0.5203218485661234
##(5개)quality로 했을 때: 0.5141324530637508

##(11개)이진분류: 0.7437590262017743
##(9개)이진분류: 0.7313802351970291
##(7개)이진분류: 0.7317928615638539
##(5개)이진분류: 0.7198266969259336

0.5141324530637508

#### data의 scale 조정

In [244]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [245]:
lin_clf = LinearSVC(max_iter=100000, random_state=42)
lin_clf.fit(X_train_scaled, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=100000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [246]:
y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)
##(11개)quality로 했을 때: 0.5289870022694451
##(9개)quality로 했을 때: 0.5302248813699195
##(7개)quality로 했을 때: 0.5242417990509594
##(5개)quality로 했을 때: 0.5230039199504848

##(11개)이진분류: 0.7404580152671756
##(9개)이진분류: 0.7439653393851867
##(7개)이진분류: 0.7435527130183619
##(5개)이진분류: 0.7412832680008252

0.5230039199504848

#### RBF 커널 적용

In [247]:
svm_clf = SVC(gamma='auto', decision_function_shape="ovr")
svm_clf.fit(X_train_scaled[:4847], y_train[:4847]) ##4847개 data로 모델 훈련

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [248]:
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)
##(11개)quality로 했을 때: 0.6075923251495771
##(9개)quality로 했을 때: 0.594800907778007
##(7개)quality로 했을 때: 0.5737569630699402
##(5개)quality로 했을 때: 0.5541572106457603

##(11개)이진분류: 0.796162574788529
##(9개)이진분류: 0.7924489374871054
##(7개)이진분류: 0.7815143387662472
##(5개)이진분류: 0.7542809985558078

0.5541572106457603

#### 교차 검증 이용한 랜덤 서치로 hyper parameter 튜닝

test 위해 1000개만 해봄

In [249]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, cv=3, n_iter=10, verbose=2, n_jobs=-1)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.9s finished
/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f31f8c38fd0>, 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f31f8c38350>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [250]:
rnd_search_cv.best_estimator_

SVC(C=7.10578688054794, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.03907162851981535,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [255]:
rnd_search_cv.best_score_
##(11개)quality로 했을 때: 0.523
##(9개)quality로 했을 때: 0.521
##(7개)quality로 했을 때: 0.516
##(5개)quality로 했을 때: 0.518

##(11개)이진분류: 0.745
##(9개)이진분류: 0.753
##(7개)이진분류: 0.749
##(5개)이진분류: 0.74

0.518

1000개만 해서 그런가 낮다.  
이제 전체에 대해 해보자

In [252]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=7.10578688054794, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.03907162851981535,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [253]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)
##(11개)quality로 했을 때: 0.5970703527955437
##(9개)quality로 했을 때: 0.5320817000206313
##(7개)quality로 했을 때: 0.5479678151433877
##(5개)quality로 했을 때: 0.5438415514751392

##(11개)이진분류: 0.8116360635444605
##(9개)이진분류: 0.7831648442335465
##(7개)이진분류: 0.7672787291107902
##(5개)이진분류: 0.7478852898700227

0.5438415514751392

In [254]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)
##(11개)quality로 했을 때: 0.5792079207920792
##(9개)quality로 했을 때: 0.5482673267326733
##(7개)quality로 했을 때: 0.557549504950495
##(5개)quality로 했을 때: 0.5544554455445545

##(11개)이진분류: 0.7803217821782178
##(9개)이진분류: 0.7648514851485149
##(7개)이진분류: 0.7660891089108911
##(5개)이진분류: 0.7561881188118812

0.5544554455445545

---

## 선형SVM회귀

In [63]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [64]:
lin_svr = LinearSVR(max_iter=10000, random_state=42)
lin_svr.fit(X_train_scaled, y_train)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=10000,
     random_state=42, tol=0.0001, verbose=0)

#### 훈련 세트에 대한 데이터 성능

MSE란?
- 손실함수: 정답에 대한 오류를 숫자로 나타내는 것
- 오답에 가까울 수록 큰 값이 나온다.
- <span style="color:red">정답에 가까울 수록 작은 값이 나온다.</span>

In [103]:
from sklearn.metrics import mean_squared_error

y_pred = lin_svr.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
mse
##(11개)quality로 했을 때:0.5565858986694731
##(11개)이진분류: 0.19523842574453554

0.19523842574453554

RMSE
- 평균 제곱근 오차
- 회귀 모델 평가시 사용
- 표준편차와 동일, 특정 수치에 대한 예측의 정확도

In [104]:
np.sqrt(mse)
##(11개)quality로 했을 때:0.7460468475032068 -- 타깃이 0~10(3~9)
##(11개)이진분류: 0.44185792484070663

0.44185792484070663

###### 타깃이 0(5이하) 또는 1(6이상)이었기 때문에 저정도의 에러는 우리의 모델을 정확하게 분류할 수 없다고 판단. 거의 찍는 수준??
오류가 감소하긴 함


## 유사도 함수를 이용한 SVM 회귀

- 유사도 함수로 RBF 커널 이용.


- 유사도 함수란? : 각 샘플이 특정 랜드마크와 얼마나 닮았는지 측정하는 것
    - 비선형 특성을 다루는 기법 중 하나로 유사도 함수로 계산한 특성을 추가한다.


- 랜드마크 선택 방법
    - 간단한 방법으로 모든 샘플 위치에 랜드마크 설정 ㅡ> __차원이 매우 커진다__
    - 차원이 매우 커지기 때문에 변환된 훈련 세트가 선형적으로 구분될 _가능성_ 이 높다.
    
    
- RBF란?
    - 방사 기저 함수(라는 이름을 가지고있다)
    

- 하이퍼파라미터 C와 gamma의 적절한 값을 찾기 위해 교차 검증을 사용한 랜덤 서치를 적용

In [105]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, cv=3, n_iter=10, verbose=2, random_state=42, n_jobs=-1)
rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   20.4s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f31fab0dd10>, 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f31fab0dc90>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [106]:
rnd_search_cv.best_estimator_

SVR(C=1.2058449429580245, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma=0.0870602087830485, kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

### 훈련 세트에서 RMSE

In [107]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)
##(11개)quality로 했을 때:0.5999828200344473
##(11개)이진분류: 0.3691439141701268

0.3691439141701268

오류가 감소하였으므로 이 모델을 test에 적용해보자

### test 세트에서 평가

In [108]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)
##(11개)quality로 했을 때:0.6670272964753355
##(11개)이진분류: 0.40441321433989735

0.40441321433989735

이정도 오류가 나왔을 때 어떤 수준인지?